In [1]:
# For computing normalized audio, spectrograms, and pitches
import os
from uberduck_ml_dev.data.get import get_mels, get_pitches
from uberduck_ml_dev.data.data import RADTTS_DEFAULTS as data_config

from uberduck_ml_dev.data.get import get
import librosa
import numpy as np
from scipy.io.wavfile import write
from datetime import datetime

In [11]:
# data_dir = "/path/to/data"
data_dir = "/usr/src/app/uberduck_ml_dev/tutorials/radtts/lj/LJSpeech/"

In [15]:
def find_rel_paths(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            yield os.path.relpath(os.path.join(root, filename), directory)

filename = 'gt.wav'  # replace with your filename
rel_path_list = list(find_rel_paths(data_dir, filename))

In [24]:
local_path_list = [os.path.join(data_dir, rel_path) for rel_path in rel_path_list]
resampled_normalized_path_list = [os.path.join(data_dir, 
                                               local_path.split('gt.wav')[0],
                                               'audio_resampledT_normalized32768T.wav') 
                                  for local_path in local_path_list]
spectrogram_path_list = [os.path.join(data_dir, local_path.split('gt.wav')[0],
                                               'spectrogram.pt') 
                                  for local_path in local_path_list]
folder_path_list = [os.path.join(data_dir, local_path.split('gt.wav')[0]) 
                                  for local_path in local_path_list]

In [ ]:
print("resample and normalize")
MAX_WAV_VALUE = 32768
sr = 22050
loading_function = lambda filename : librosa.load(filename, sr = 22050)[0]
function_ = lambda x : np.asarray((x / np.abs(x).max()) * (MAX_WAV_VALUE - 1), dtype = np.int16)
saving_function = lambda data, filename  : write(filename, 22050, data) # must be in this order

print(datetime.now())
get(function_, loading_function, saving_function, local_path_list, resampled_normalized_path_list, False)
print(datetime.now())

In [ ]:
print("compute pitches")
get_pitches(resampled_normalized_path_list, data_config, folder_path_list, method = 'radtts')

In [ ]:
print("compute mels")
get_mels(resampled_normalized_path_list, data_config, spectrogram_path_list)